This is Nirs implementation of the TACo algorithm in python, aswell as his enhancement of the algorithm called RankedTACo. 

You can run this program and see the comparision in the number of steps completed by the RankedTACo vs normal Taco, aswell as see the gap in optimality. 

In [1]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 5.3 MB 5.8 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from collections import Counter
import math


In [3]:
def TACo(n, m, C, d0, gamma, epsilon, b):
    O = np.zeros((n, m), dtype=float)  # Step 1
    P = np.zeros((n, m), dtype=float)  # Step 2
    selections = []  # Step 3
    is_converged = False  # Step 4
    d = d0  # Step 5
    recorded_states = set()  # Step 6
    steps = 0

    while not is_converged:  # Step 7
        steps += 1
        if steps >= 10000:
            print("TACO failed over 10000 steps")
            return -1, 10000

        current_selections = []
        for i in range(n):  # Step 8
            J = b[i, i] * (O[i] - P[i]) - C[i]  # Steps 9-11
            j_star = np.argmax(J)  # Step 13

            P[i, j_star] += n * d  # Step 15
            O[:, j_star] += d  # Step 16

            current_selections.append(j_star)
            selections.append(j_star)  # Step 17

            current_state = (tuple(map(tuple, O - P)), i)  # Store (O - P) and agent i
            if current_state in recorded_states:  # Step 18
                d *= gamma  # Step 20
                recorded_states.clear()  # Step 21
            else:
                recorded_states.add(current_state)  # Step 29

        if len(set(current_selections)) == 1:
            is_converged = True  # Immediate termination if all agents select the same choice
        elif all(np.max(b[i, i] * (O[i] - P[i]) - C[i]) - np.min(b[i, i] * (O[i] - P[i]) - C[i]) < epsilon for i in range(n)):
            is_converged = True  # Steps 22-26

    print("TACo Number of steps:", steps)
    return Counter(selections).most_common(1)[0][0], steps  # Step 31

In [4]:
import numpy as np
from collections import Counter

def rankedTACo(n, m, C, d0, gamma, epsilon, b):
    O = np.zeros((n, m), dtype=float)
    P = np.zeros((n, m), dtype=float)
    selections = []
    is_converged = False
    d = d0
    recorded_states = set()
    steps_ranked = 0

    while not is_converged:
        steps_ranked += 1

        if steps_ranked >= 10000:
            print("RankedTACO Failed over 10000 steps")
            return -1, 10000

        current_selections = []
        J_matrix = np.zeros((n, m), dtype=float)  # Initialize J as a matrix

        for i in range(n):
            J = b[i, i] * (O[i] - P[i]) - C[i]
            ranked_choices = np.sort(J)
            j_star = np.argmax(J)
            current_selections.append(j_star)
            J_matrix[i] = J

            for index_choices in J:
                index = int(np.where(J == index_choices)[0])
                ranked_index = int(np.where(ranked_choices == index_choices)[0])
                P[i, index] += n * d * (1 + ranked_index)
                O[:, index] += d * (1 + ranked_index)

            # Cycle detection inside loop
            current_state = (tuple(map(tuple, J_matrix)), i)  # Store J matrix and player index
            if current_state in recorded_states:
                d *= gamma
                recorded_states.clear()
            else:
                recorded_states.add(current_state)

        selections.extend(current_selections)
        #print(current_selections)
        if len(set(current_selections)) == 1:
            is_converged = True

        if all(np.max(J_matrix[i]) - np.min(J_matrix[i]) < epsilon for i in range(n)):
            is_converged = True

    print("Ranked TACo number of steps:", steps_ranked)
    return Counter(selections[-n:]).most_common(1)[0][0], steps_ranked


In [ ]:
#This is a randomized simulation
# AT the end it outputs the opt_gap

n = 8




d0 = 5.0
gamma = 0.9
epsilon = 1
rmse = 0
count = 0
opt_gap = 0

ranked_sum = 0
normal_sum = 0

average_steps_taco = 0
average_setps_ranked = 0

diverged_normal = 0
diverged_ranked = 0
average_opt_gap = 0
for i in range(0, 300):

  m = np.random.randint(10, 20)
  #print("Progress", i, "N", n, "M", m)

  diagonal_values = np.random.rand(n)
  b = np.diag(diagonal_values)
  C = np.random.rand(n,m)*50

  #C = np.array([[10 ,4], [7, 9]])
  #b = np.array([[.8 ,0], [0, 1.2]])

  #print("NORMAL")

  result_n, taco_steps = TACo(n, m, C, d0, gamma, epsilon,b)
  result_r, ranked_steps = rankedTACo(n, m, C, d0, gamma, epsilon,b)

  if result_n != -1 and result_r != -1:


    #print(f"The most common selection is: {result_r}")

    for players in range(0, n):
      ranked_sum += C[players][result_r]
      normal_sum += C[players][result_n]

    rmse += ((result_r-result_n)/max(result_r, result_n))**2
    count+=1



    average_setps_ranked += ranked_steps
    average_steps_taco += taco_steps
  else:
    average_setps_ranked += ranked_steps
    average_steps_taco += taco_steps
    if taco_steps >= 10000:
      diverged_normal += 1
    if ranked_steps >= 10000:
      diverged_ranked += 1



opt_gap = ((ranked_sum/normal_sum) - 1)

print("RMSE", math.sqrt(rmse/count))
print("OPT Gap", opt_gap)
print("Average Steps Ranked TACo", average_setps_ranked/count)
print("Average Steps  TACo", average_steps_taco/count)
print("Number of times ranked diverged:", diverged_ranked)
print("Number of times normal diverged:", diverged_normal)







Progress 0 N 8 M 16
TACo Number of steps: 14
Ranked TACo number of steps: 11
Progress 1 N 8 M 17
TACo Number of steps: 10
Ranked TACo number of steps: 3
Progress 2 N 8 M 16
TACo Number of steps: 12
Ranked TACo number of steps: 6
Progress 3 N 8 M 12
TACo Number of steps: 5
Ranked TACo number of steps: 6
Progress 4 N 8 M 13
TACo Number of steps: 11
Ranked TACo number of steps: 10
Progress 5 N 8 M 17
TACo Number of steps: 21
Ranked TACo number of steps: 4
Progress 6 N 8 M 14
TACo Number of steps: 7
Ranked TACo number of steps: 14
Progress 7 N 8 M 15


/var/folders/3x/y4c80np166bb8ng4y4yx6zkr0000gn/T/ipykernel_37655/2923448490.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  index = int(np.where(J == index_choices)[0])
/var/folders/3x/y4c80np166bb8ng4y4yx6zkr0000gn/T/ipykernel_37655/2923448490.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ranked_index = int(np.where(ranked_choices == index_choices)[0])


TACO failed over 10000 steps
Ranked TACo number of steps: 194
Progress 8 N 8 M 14
TACo Number of steps: 12
Ranked TACo number of steps: 4
Progress 9 N 8 M 13
TACO failed over 10000 steps
Ranked TACo number of steps: 95
Progress 10 N 8 M 19
TACo Number of steps: 17
Ranked TACo number of steps: 8
Progress 11 N 8 M 15
TACo Number of steps: 17
Ranked TACo number of steps: 7
Progress 12 N 8 M 14
TACo Number of steps: 11
Ranked TACo number of steps: 5
Progress 13 N 8 M 12
TACo Number of steps: 19
Ranked TACo number of steps: 2
Progress 14 N 8 M 15


/var/folders/3x/y4c80np166bb8ng4y4yx6zkr0000gn/T/ipykernel_37655/3471338793.py:51: RuntimeWarning: invalid value encountered in scalar divide
  rmse += ((result_r-result_n)/max(result_r, result_n))**2


TACO failed over 10000 steps
Ranked TACo number of steps: 15
Progress 15 N 8 M 14
TACo Number of steps: 11
Ranked TACo number of steps: 4
Progress 16 N 8 M 16
TACo Number of steps: 12
Ranked TACo number of steps: 7
Progress 17 N 8 M 12
TACo Number of steps: 9
Ranked TACo number of steps: 7
Progress 18 N 8 M 14
TACo Number of steps: 4
Ranked TACo number of steps: 5
Progress 19 N 8 M 12
TACo Number of steps: 10
Ranked TACo number of steps: 5
Progress 20 N 8 M 18
TACo Number of steps: 121
Ranked TACo number of steps: 20
Progress 21 N 8 M 10
TACo Number of steps: 23
Ranked TACo number of steps: 5
Progress 22 N 8 M 14
TACo Number of steps: 13
Ranked TACo number of steps: 13
Progress 23 N 8 M 12
TACo Number of steps: 12
Ranked TACo number of steps: 10
Progress 24 N 8 M 10
TACO failed over 10000 steps
Ranked TACo number of steps: 7
Progress 25 N 8 M 14
TACo Number of steps: 11
Ranked TACo number of steps: 9
Progress 26 N 8 M 15
TACO failed over 10000 steps
Ranked TACo number of steps: 54
Prog

In [ ]:
n = 4
m = 24
d0 = 1
epsilon=1
gamma = .9
diagonal_values = np.random.rand(n)
b = np.diag(diagonal_values)*2
C = np.random.rand(n,m)
#result_r, ranked_steps = rankedTACo(n, m, C, d0, gamma, epsilon,b)
result_n, taco_steps = TACo(n, m, C, d0, gamma, epsilon,b)

print("RESULT", result_n)